# 程序说明
时间：2016年11月17日

名称：迁移学习例程

说明：该程序演示将一个预训练好的模型在新数据集上重新fine-tuning的过程。我们冻结卷积层，只调整全连接层。
1. 在MNIST数据集上使用前五个数字[0...4]训练一个卷积网络。
2. 在后五个数字[5...9]用卷积网络做分类，冻结卷积层并且微调全连接层

数据集：MNIST

## 1.加载keras模块

In [1]:
from __future__ import print_function
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


## 2.变量初始化

In [2]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# 输入图像的维度
img_rows, img_cols = 28, 28
# 使用卷积滤波器的数量
nb_filters = 32
# 用于max pooling的pooling面积的大小
pool_size = 2
# 卷积核的尺度
kernel_size = 3

if K.image_dim_ordering() == 'th':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)


## 3.准备数据
将MNIST划分为两个数据集，分别用于预训练模型和迁移学习模型。

In [3]:
# 数据，在训练和测试数据集上混洗和拆分
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

### 模型的训练函数

In [4]:
def train_model(model, train, test, nb_classes):
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

## 4.建立模型
### 构建卷积层（特征层）和全连接层（分类层）

In [5]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

## 5.训练预训练模型用于5个数字[0...4]的分类任务

In [6]:
# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 5s - loss: 0.2148 - acc: 0.9349 - val_loss: 0.0398 - val_acc: 0.9868
Epoch 2/5
30596/30596 [==============================] - 4s - loss: 0.0631 - acc: 0.9810 - val_loss: 0.0234 - val_acc: 0.9924
Epoch 3/5
30596/30596 [==============================] - 4s - loss: 0.0457 - acc: 0.9869 - val_loss: 0.0192 - val_acc: 0.9926
Epoch 4/5
30596/30596 [==============================] - 4s - loss: 0.0367 - acc: 0.9885 - val_loss: 0.0150 - val_acc: 0.9946
Epoch 5/5
30596/30596 [==============================] - 4s - loss: 0.0304 - acc: 0.9911 - val_loss: 0.0157 - val_acc: 0.9947
Training time: 0:00:24.128250
Test score: 0.0156876081334
Test accuracy: 0.994746059545


## 6.冻结特征层

In [7]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

## 7.fine-tuning分类层

In [8]:
# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 3s - loss: 0.3601 - acc: 0.8942 - val_loss: 0.0906 - val_acc: 0.9704
Epoch 2/5
29404/29404 [==============================] - 2s - loss: 0.1231 - acc: 0.9625 - val_loss: 0.0584 - val_acc: 0.9809
Epoch 3/5
29404/29404 [==============================] - 3s - loss: 0.0917 - acc: 0.9732 - val_loss: 0.0480 - val_acc: 0.9837
Epoch 4/5
29404/29404 [==============================] - 3s - loss: 0.0753 - acc: 0.9777 - val_loss: 0.0387 - val_acc: 0.9856
Epoch 5/5
29404/29404 [==============================] - 3s - loss: 0.0654 - acc: 0.9805 - val_loss: 0.0347 - val_acc: 0.9875
Training time: 0:00:15.838667
Test score: 0.0347462616509
Test accuracy: 0.98745114163
